In [1]:
import numpy as np

TODOs:
- [ ] load circuit from frontend
- [ ] circuit data structure
- [ ] algorithm to convert circuit into digraph
- [ ] algorithm: circuit DS to incidence matrices
- [ ] load voltage source values and current source values from frontend
- [ ] algorithm: generate deflation matrix D0 (determine ground node)
- [ ] algorithm: generate particular solution vector, pg


## step 1~2. Number of nodes and elements

In [2]:
# TODO: pass from front-end

n_g = 7 # number of nodes in the circuit

m_r = 8 # number of resistors in circuit
m_v = 2 # number of voltage sources in circuit
m_i = 2 # number of current sources in circuit
m = m_r + m_v + m_i # total number of elements in circuit

## step 3. incidence matrix (edges, which were originally elements)

In [3]:
# TODO: algorithm to generate the incidence matrix

# resistor subblock of the incidence matrix
# -1: in; 1: out; 0: not connected
Arg = np.array([
    [1, 0, 0, 0, 0, 0, -1],
    [-1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, -1],
    [0, 1, -1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, -1, 0],
    [0, 0, 0, 0, 0, 1, -1],
    [0, 0, 1, -1, 0, 0, 0],
    [0, 0, 0, 0, 1, -1, 0]
])

In [4]:
# voltage source subblock of the incidence matrix
Avg = np.array([
    [0, 1, 0, 0, 0, 0, -1],
    [0, 0, 0, 1, -1, 0, 0]
])

In [5]:
# current source subblock of the incidence matrix
Aig = np.array([
    [0, -1, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, -1, 0, 1]
])

In [6]:
# full incidence matrix
Ag = np.vstack((Arg, Avg, Aig))
Ag

array([[ 1,  0,  0,  0,  0,  0, -1],
       [-1,  1,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0, -1],
       [ 0,  1, -1,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  1, -1],
       [ 0,  0,  1, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  1, -1,  0],
       [ 0,  1,  0,  0,  0,  0, -1],
       [ 0,  0,  0,  1, -1,  0,  0],
       [ 0, -1,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  1]])

## step 5. state the entire set of circuit equations

In [7]:
# TODO: read from data

# known ideal values provided by voltage sources
Vv = np.array([[5], [5]])
# known ideal values provided by current sources
Ii = np.array([[2.5], [2.5]])
Vv, Ii

(array([[5],
        [5]]),
 array([[2.5],
        [2.5]]))

In [8]:
# resistor matrix: diagonal matrix with the resistances
# in this case, all resistances are 1 Ohm
R = np.diag(np.ones(m_r))
R

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [9]:
# Conductance matrix G: inverse of the resistance matrix
# given resistance R, conductance G = 1/R
G = np.linalg.inv(R)
G

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

## step. 7 Ground the circuit
The ground node is the reference node. The voltage at the ground node is 0V.

In [10]:
# TODO algorithmically generate the D0 matrix

D0 = np.eye(n_g)
D0 = D0[:, :-1]  # remove the last column (ground node)
D0

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0.]])

In [11]:
# Ground the circuit equations using deflation
Ar0 = Arg @ D0
Av0 = Avg @ D0
Ai0 = Aig @ D0
A0 = Ag @ D0

Ar0, Av0, Ai0, A0

(array([[ 1.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  1., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1., -1.]]),
 array([[ 0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1., -1.,  0.]]),
 array([[ 0., -1.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  0.]]),
 array([[ 1.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  1., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1., -1.],
        [ 0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1., -1.,  0.],
        [ 0., -1.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  0.]]))

## step 8. State the Grounded Circuit Equations

## Step 9. Identify essential nodes, nonessential nodes, and supernodes

## Step 10. Eliminate the node dependencies that arise due to voltage sources
Create a particular solution to GLSP

In [12]:
# TODO: algorithmically generate the pg (particular solution) vector
pg = np.array([[0], [Vv[0, 0]], [0], [0], [-Vv[1, 0]], [0], [0]])
pg

array([[ 0],
       [ 5],
       [ 0],
       [ 0],
       [-5],
       [ 0],
       [ 0]])

In [13]:
# TODO: algorithmically create the deflation matrix
# Create the deflation matrix for the entire set of voltage source KVLs
Dvg = np.array([
    [1, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 0, 0, 0]
])
Dvg

array([[1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0]])

In [14]:
D = D0.T @ Dvg
Ar = Ar0 @ D
Ai = Ai0 @ D
x0 = D0.T @ pg
b = -Ar0 @ x0

In [15]:
K = Ar.T @ G @ Ar
f = Ar.T @ G @ b - Ai.T @ Ii

In [16]:
u, _, _, _ = np.linalg.lstsq(K, f, rcond=None) # solve the linear system Ku = f
u0 = x0 + D @ u # compute the solution for the original system

In [18]:
print("Node Voltages:")
u0 = np.round(u0, 3)
u0

Node Voltages:


array([[2.5 ],
       [5.  ],
       [3.75],
       [5.  ],
       [0.  ],
       [1.25]])